In [0]:
%sql
CREATE CONNECTION IF NOT EXISTS DB_Geekcoder_Conn
TYPE HTTP
OPTIONS (
  host = 'https://earthquake.usgs.gov',
  port = 443,
  base_path = '/earthquakes/feed/v1.0/',
  bearer_token = 'na' )

In [0]:
%py
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

conn = w.connections.get("DB_Geekcoder_Conn")
base_url = f"{conn.options['host']}{conn.options['base_path']}"

In [0]:
dbutils.widgets.text('catalog_name','db_project_geekcoders', 'db_project_geekcoders')
catalog_name=dbutils.widgets.get('catalog_name').strip()

In [0]:
spark.sql(
    f"use catalog {catalog_name}" )
spark.sql(
    "use schema bronze" )
spark.sql(
    "create volume if not exists earthquake_data" )  

In [0]:
import requests
import json
import datetime

url = f"{base_url}/summary/all_day.geojson"
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Error in getting data from {url}")
data = response.json()
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
dbutils.fs.put(
    f"/Volumes/{catalog_name}/bronze/earthquake_data/earthquake_data_{current_date}.json",
    json.dumps(data),
    overwrite=True,
)

In [0]:
%sql
use catalog db_project_geekcoders;
use schema bronze;
create volume if not exists earthquake_data;